In [1]:
## Data Initializing

In [1]:
import pandas as pd
import numpy as np
import Newton_optimization as Newton

In [2]:
# model parameters:

kGamma = 0.5
kMu = 1.0 / 4

## Data Preprocessing

In [3]:
import DataHandler as dh
handler = dh.DataHandler()
graph_data = handler.GetGraphData('EMA_net.tntp', [0, 1, 2, 4])

kNodesNumber = graph_data['kNodesNumber']
kLinksNumber = graph_data['kLinksNumber']

kMaxPathLength = 100

graph_table = graph_data['graph_table']
graph_table
#graph_table.dtypes

,Init node,Term node,Capacity,Free Flow Time
0,1,3,4938.061523,0.238965
1,3,1,5254.128418,0.240297
2,1,7,7309.824707,0.222813
3,7,1,7388.626953,0.218902
4,1,9,1164.374878,0.402046
5,9,1,1076.221558,0.403219
6,2,3,885.069885,0.106992
7,3,2,879.337646,0.108032
8,3,6,5963.515137,0.109499
9,6,3,6000.000000,0.107665


In [4]:
#graph correspondences
graph_correspondences = handler.GetGraphCorrespondences('EMA_trips.tntp')
graph_correspondences

{1: {1: 0.0,
  2: 63.802849,
  3: 471.81948,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 918.488499,
  8: 0.0,
  9: 0.0,
  10: 0.0,
  11: 0.0,
  12: 0.0,
  13: 0.0,
  14: 0.0,
  15: 0.0,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: 0.0,
  20: 0.0,
  21: 28.772957,
  22: 31.681824,
  23: 0.0,
  24: 0.0,
  25: 0.0,
  26: 0.0,
  27: 0.0,
  28: 0.0,
  29: 29.681628,
  30: 8.606275,
  31: 8.606275,
  32: 8.606275,
  33: 8.606275,
  34: 0.0,
  35: 0.0,
  36: 11.722161,
  37: 20.428002,
  38: 20.428002,
  39: 20.428002,
  40: 20.428002,
  41: 0.0,
  42: 23.394305,
  43: 0.0,
  44: 0.0,
  45: 0.0,
  46: 0.0,
  47: 0.0,
  48: 14.575279,
  49: 8.505481,
  50: 9.077854,
  51: 9.077854,
  52: 4.484028,
  53: 4.452338,
  54: 1.875118,
  55: 0.0,
  56: 0.0,
  57: 0.0,
  58: 0.0,
  59: 2.02354,
  60: 2.02354,
  61: 1.880171,
  62: 1.93676,
  63: 1.93676,
  64: 1.93676,
  65: 1.93676,
  66: 1.93676,
  67: 1.948398,
  68: 0.0,
  69: 1.965538,
  70: 0.0,
  71: 0.0,
  72: 0.0,
  73: 0.0,
  74: 0.0},
 2: {1: 44.610847,


## Graph Constructing

In [5]:
# Attention: as shown on the table above
# nodes indexed from 1 to ...
# edges indexed from 0 to ...
import TransportGraph as tg
graph = tg.TransportGraph(graph_data)
print(graph.Nodes())
print(graph.Successors(1))
#an_graph = copy(graph)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]
[9, 3, 7]


In [6]:
import Oracles
import DataHandler as dh
handler = dh.DataHandler()
graph_data = handler.GetGraphData('my_example_net.tntp', [0, 1, 2, 4])

kNodesNumber = graph_data['kNodesNumber']
kLinksNumber = graph_data['kLinksNumber']

kMaxPathLength = 100

graph_table = graph_data['graph_table']
graph_table
#graph_table.dtypes


,Init node,Term node,Capacity,Free Flow Time
0,1,2,4938.061523,0.238965
1,2,3,5254.128418,0.240297
2,3,4,7309.824707,0.222813
3,4,1,7388.626953,0.218902


In [7]:
import TransportGraph as tg
graph = tg.TransportGraph(graph_data)
print(graph.Nodes())
print(graph.Successors(1))
#an_graph = copy(graph)
graph_correspondences = handler.GetGraphCorrespondences('my_example_trips.tntp')
graph_correspondences[1]

[1, 2, 3, 4]
[2]


{4: 5.0}

In [9]:
automatic_oracle = Oracles.AutomaticOracle(source = 1, drain = 4, graph = graph,
                                           source_correspondences = graph_correspondences[1])

automatic_oracle.grad(np.array([1, 1, 1, 1]))

NameError: global name 'BetaDerivativeOfBetaFunction' is not defined

In [ ]:
class UniversalTriangularMethod:
    def __init__(self, source, drain, graph, source_correspondences):
    self.graph = graph #should it be here? 
    self.source = source
    self.drain = drain
    self.source_correspondences = source_correspondences
    self.A_values = - np.inf * np.ones((kMaxPathLength, kNodesNumber))
    self.B_values = - np.inf * np.ones((kMaxPathLength, kNodesNumber))
    self.t_current = None

## Automatic Differentiation

In [153]:
# from scipy.special import expit
from collections import defaultdict
from scipy.misc import logsumexp
from scipy.special import expit

class BaseOracle(object):
    """
    Base class for implementation of oracles.
    """
    def func(self, x):
        """
        Computes the value of function at point x.
        """
        raise NotImplementedError('Func oracle is not implemented.')

    def grad(self, x):
        """
        Computes the gradient at point x.
        """
        raise NotImplementedError('Grad oracle is not implemented.')


class AutomaticOracle(BaseOracle):
    """
    Oracle for automatic calculations of function \Psi (t)
    """

    def __init__(self, source, drain, graph, source_correspondences):
        self.graph = graph #should it be here? 
        self.source = source
        self.drain = drain
        self.source_correspondences = source_correspondences
        self.A_values = - np.inf * np.ones((kMaxPathLength, kNodesNumber))
        self.B_values = - np.inf * np.ones((kMaxPathLength, kNodesNumber))
        self.t_current = None
        
        
    # this function should be called before calling func and grad    
    def CalculatePreliminaryData(self):
        for term_vertex in self.graph.Nodes():
            self.CalculateTermVertexABvalues(self, term_vertex)
            
    def Reset(self, t_parameter):
        self.A_values = - np.inf * np.ones((kMaxPathLength, kNodesNumber))
        self.B_values = - np.inf * np.ones((kMaxPathLength, kNodesNumber))
        sefl.t_current = t_parameter
        self.CalculatePreliminaryData()

    def func(self, t_parameter):
        if t_current != t_parameter:
            self.Reset(t_parameter)
            
        return GetArrayValues(self.B_values, kMaxPathLength, self.drain)

    def grad(self, t_parameter):
        if t_current != t_parameter:
            self.Reset(t_parameter)
        
        #calculating beta_d_beta_values, beta_d_alpha_values
        beta_d_beta_values = np.zeros((kMaxPathLength, kNodesNumber))
        beta_d_alpha_values = np.zeros((kMaxPathLength, kNodesNumber))
        for path_length in range(kMaxPathLength, 1, -1):
            SetArrayValues(BetaDerivativeOfBetaFunction(path_length), beta_d_beta_values, path_length)
            SetArrayValues(AlphaDerivativeOfBetaFunction(path_length), beta_d_alpha_values, path_length)
        
        #calculating psi_d_beta_values
        psi_d_beta_values = np.zeros((kMaxPathLength, kNodesNumber))
        correspondences_target_vertices = graph_correspondences[1].keys()
        SetArrayValues(self.source_correspondences.values(), psi_d_beta_values,
                       kMaxPathLength, self.source_correspondences.keys())
        #TODO!!! above Correspondences for ALL nodes
        for path_length in range(kMaxPathLength - 1, 0, -1):
            psi_d_beta_new_values = GetArrayValues(psi_d_beta_values, path_length + 1) * 
                                    GetArrayValues(beta_d_beta_values, path_length + 1)
            SetArrayValues(psi_d_beta_new_values, psi_d_beta_values, path_length)
        
        #calculating psi_d_alpha_values
        #this place is very unoptimized. Too many cycles. Need numpy style calculating
        #try reduce cycle of nodes.Need vectorized function(nodes)
        psi_d_alpha_values = np.zeros((kMaxPathLength, kNodesNumber))
        SetArrayValues(0.0, psi_d_alpha_values_, kMaxPathLength)
        for path_length in range(kMaxPathLength - 1, 0, -1):
            first_terms = GetArrayValues(psi_d_beta_values, path_length) * 
                          GetArrayValues(beta_d_alpha_values, path_length)
                
            second_terms = np.zeros(kNodesNumber)
            for node in self.graph.Nodes():
                second_term_value = np.sum(self.GetArrayValues(psi_d_alpha_values, path_length + 1,
                                                               self.graph.Successors(node))
                                           * self.AlphaDerivativeOfAlphaFunctions(path_length + 1, node))
                SetArrayValues(second_term_value, second_terms, path_length, node)
            
            SetArrayValues(first_terms + second_terms, psi_d_alpha_values_, path_length)
        
        #calculating gradient
        gradient_vector = np.zeros(kLinksNumber)
        for edge_index in self.graph.Edges():
            gradient_vector[edge_index] = np.sum(psi_d_alpha_values * 
                                                 self.TimeDerivativeOfAlphaFunctions(edge_index))
        
        return gradient_vector
    
    
    def TimeDerivativeOfAlphaFunctions(self, edge_index):
        derivatives = np.zeros((kMaxPathLength, kNodesNumber))
        for path_length in range(1, kMaxPathLength + 1):
            for term_vertex in self.graph.Nodes():
                edges_indices = self.graph.InEdges(term_vertex)

                if edge_index not in edges_indices:
                    return 0.0

                alpha_values = GetArrayValues(self.A_values, path_length - 1,
                                              self.graph.Predecessors(term_vertex))

                derivative_value = - 1.0/np.sum(np.exp((alpha_values - self.t_current[edges_indices] - 
                                                        alpha_values[edges_indices.index(edge_index)] + 
                                                        self.t_current[edge_index]) / kGamma))
                SetArrayValues(derivative_value, derivatives, path_length, term_vertex)
        return derivatives
        
        
    def AlphaDerivativeOfAlphaFunctions(self, path_length, deriv_term_vertex):
        derivatives = []
        for term_vertex in self.graph.Successors(deriv_term_vertex):
            edges_indices = self.graph.InEdges(term_vertex)
            freeflow_time_values = self.t_current[edges_indices]

            deriv_edge_index = transport_graph[self.source][deriv_term_vertex]['edge_index']
            assert(edge_index in edges_indices)

            alpha_values = GetArrayValues(self.A_values, path_length - 1, self.graph.Predecessors(term_vertex))

            derivative_value =  1.0/np.sum(np.exp((alpha_values - freeflow_time_values - 
                                                   alpha_values[edges_indices.index(deriv_edge_index)] + 
                                                   self.t_current[deriv_edge_index]) / kGamma))
            derivatives.append(derivative_value)
        return np.array(derivatives)
    
    def BetaDerivativeOfBetaFunction(self, path_length):
        alpha_values = GetArrayValues(self.A_values, path_length)
        beta_values = GetArrayValues(self.B_values, path_length - 1)
        return expit((beta_values - alpha_values) / kGamma)
    
    def AlphaDerivativeOfBetaFunction(self, path_length):
        alpha_values = GetArrayValues(self.A_values, path_length)
        beta_values = GetArrayValues(self.B_values, path_length - 1)
        return expit((alpha_values - beta_values) / kGamma)

    
    def CalculateTermVertexABvalues(self, term_vertex):
        initial_values = - 1.0 * self.t_current[self.graph.OutEdges(self.source)]
        SetArrayValues(initial_values, self.A_values, 1, self.graph.Successors(self.source))
        SetArrayValues(initial_values, self.B_values, 1, self.graph.Successors(self.source))
            
        for path_length in range(2, kMaxPathLength + 1):
            alpha = kGamma * \
                    logsumexp(1.0 / kGamma * 
                              (GetArrayValues(self.A_values, path_length - 1, self.graph.Predecessors(term_vertex))
                               - self.t_current[graph.InEdges(term_vertex)]))
                
            beta = kGamma * \
                    logsumexp([1.0 / kGamma * GetArrayValues(self.B_values, path_length - 1, term_vertex),
                               1.0 / kGamma * GetArrayValues(self.A_values, path_length, term_vertex)]) 
                
            SetArrayValues(alpha, self.A_values, path_length, term_vertex)
            SetArrayValues(beta, self.B_values, path_length, term_vertex)
                

                
                
def SetArrayValues(values, array, path_length, vertices_list = None):
    if vertices_list:
        array[path_length - 1][np.array(vertices_list) - 1] = np.array(values)
    else:
        array[path_length - 1][:] = np.array(values)

def GetArrayValues(array, path_length, vertices_list = None):
    if vertices_list:
        res = array[path_length - 1][np.array(vertices_list) - 1]
    else:
        res = array[path_length - 1][:]
    return res

In [20]:
ex = np.arange(1, 11).reshape(2,5)
print(ex)

def GetArrayValues(array, path_length, vertices_list = None):
    if vertices_list:
        res = array[path_length - 1][np.array(vertices_list) - 1]
    else:
        res = array[path_length - 1][:]
    return res

def SetArrayValues(values, array, path_length, vertices_list = None):
    if vertices_list:
        array[path_length - 1][np.array(vertices_list) - 1] = np.array(values)
    else:
        array[path_length - 1][:] = np.array(values)

SetArrayValues(100, ex, 2, [3])
print(ex)
SetArrayValues(100, ex, 1)
print(ex)


[[ 1  2  3  4  5]
 [ 6  7  8  9 10]]
[[  1   2   3   4   5]
 [  6   7 100   9  10]]
[[100 100 100 100 100]
 [  6   7 100   9  10]]


In [41]:
a = range(3, 1, -1)
a.append(0)
b = np.array(a)
c = np.copy(b)
print(b)
np.sum(b * c)

[3 2 0]


13

In [11]:
(not bool(1)) and bool(1)

False